In [1]:
# Import all the necessary libraries
import ipywidgets as widgets
from IPython.display import display
import requests
import os
import pandas as pd
import numpy as np

import warnings
# filter warnings
warnings.filterwarnings('ignore')

## 1. RNASeq Dataset

##### RNASeq Normalized is a first dataset that I am using. So, read RNASeq dataset using pandas and ignore few errors.

In [2]:
RNASeq = pd.read_csv('data/GBM-TP/RNASeq.txt', sep='\s+', error_bad_lines=False, index_col=False, dtype='unicode')
RNASeq

,HybridizationREF,TCGA-02-0047-01A-01R-1849-01,TCGA-02-0055-01A-01R-1849-01,TCGA-02-2483-01A-01R-1849-01,TCGA-02-2485-01A-01R-1849-01,TCGA-02-2486-01A-01R-1849-01,TCGA-06-0125-01A-01R-1849-01,TCGA-06-0125-02A-11R-2005-01,TCGA-06-0129-01A-01R-1849-01,TCGA-06-0130-01A-01R-1849-01,...,TCGA-41-3915-01A-01R-1850-01,TCGA-41-4097-01A-01R-1850-01,TCGA-41-5651-01A-01R-1850-01,TCGA-76-4925-01A-01R-1850-01,TCGA-76-4926-01B-01R-1850-01,TCGA-76-4927-01A-01R-1850-01,TCGA-76-4928-01B-01R-1850-01,TCGA-76-4929-01A-01R-1850-01,TCGA-76-4931-01A-01R-1850-01,TCGA-76-4932-01A-01R-1850-01
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.4604,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,?|100133144,3.0732,0.0000,2.9467,15.9952,5.5610,9.9246,3.3758,22.3207,0.6395,...,8.2279,15.2648,1.2190,12.2160,9.5716,2.4096,10.1709,8.4757,6.1600,0.0000
2,?|100134869,6.7611,15.6973,13.9398,14.9571,4.8049,11.6684,3.3862,16.8239,2.9706,...,10.2174,13.8987,10.3611,17.3596,13.9689,6.4257,6.4812,3.2496,7.9576,9.9502
3,?|10357,54.7036,31.3945,60.3441,91.8238,62.5366,52.1491,41.6125,76.2289,35.0696,...,35.5731,27.6285,70.0838,108.1354,70.5556,43.0201,42.4584,62.2278,58.8188,60.1990
4,?|10431,886.3210,885.7744,1234.8285,470.0000,546.3415,391.9332,419.2458,642.3833,774.1104,...,512.5165,593.2464,507.6947,531.9331,588.0414,620.8835,533.7423,668.9001,458.3529,499.5025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,3867.9738,11481.4296,5066.8074,5090.9524,7800.0000,14516.6022,6251.2354,3838.6419,11495.6163,...,6793.1489,4724.4820,3173.5487,10100.3000,5263.6535,7469.0763,16936.8975,2922.3897,7212.2353,9608.4577
20527,ZZEF1|23140,1397.0927,606.0266,832.0844,1139.5238,627.4390,1350.9880,1396.6190,1963.4101,793.7081,...,948.6166,1670.7598,760.6278,752.6790,1092.7495,1244.1767,1024.9781,1862.6466,1561.4118,795.0249
20528,ZZZ3|26009,603.5834,718.1500,705.4354,700.4762,527.4390,730.0876,664.7594,1114.1786,900.4642,...,778.6561,599.3860,537.5590,910.4158,746.2335,541.3655,512.7082,632.6075,568.9412,748.7562
20529,psiTPTE22|387590,6.7611,12.3336,398.9446,15.7143,69.5122,6.1112,17.6853,336.2314,17.0191,...,14.4928,13.8143,11.5801,12.0017,10.3578,13.6546,7.8878,10.6086,5.1765,0.0000


##### Slicing sample ids upto 16 characters

In [3]:
SlicedId = RNASeq.columns.str.slice(0, 16).unique()

# Adding new headers, which is sliced headers
RNASeq = RNASeq.set_axis(SlicedId, axis=1, inplace=False) 
RNASeq

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0125-01A,TCGA-06-0125-02A,TCGA-06-0129-01A,TCGA-06-0130-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.4604,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,?|100133144,3.0732,0.0000,2.9467,15.9952,5.5610,9.9246,3.3758,22.3207,0.6395,...,8.2279,15.2648,1.2190,12.2160,9.5716,2.4096,10.1709,8.4757,6.1600,0.0000
2,?|100134869,6.7611,15.6973,13.9398,14.9571,4.8049,11.6684,3.3862,16.8239,2.9706,...,10.2174,13.8987,10.3611,17.3596,13.9689,6.4257,6.4812,3.2496,7.9576,9.9502
3,?|10357,54.7036,31.3945,60.3441,91.8238,62.5366,52.1491,41.6125,76.2289,35.0696,...,35.5731,27.6285,70.0838,108.1354,70.5556,43.0201,42.4584,62.2278,58.8188,60.1990
4,?|10431,886.3210,885.7744,1234.8285,470.0000,546.3415,391.9332,419.2458,642.3833,774.1104,...,512.5165,593.2464,507.6947,531.9331,588.0414,620.8835,533.7423,668.9001,458.3529,499.5025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,3867.9738,11481.4296,5066.8074,5090.9524,7800.0000,14516.6022,6251.2354,3838.6419,11495.6163,...,6793.1489,4724.4820,3173.5487,10100.3000,5263.6535,7469.0763,16936.8975,2922.3897,7212.2353,9608.4577
20527,ZZEF1|23140,1397.0927,606.0266,832.0844,1139.5238,627.4390,1350.9880,1396.6190,1963.4101,793.7081,...,948.6166,1670.7598,760.6278,752.6790,1092.7495,1244.1767,1024.9781,1862.6466,1561.4118,795.0249
20528,ZZZ3|26009,603.5834,718.1500,705.4354,700.4762,527.4390,730.0876,664.7594,1114.1786,900.4642,...,778.6561,599.3860,537.5590,910.4158,746.2335,541.3655,512.7082,632.6075,568.9412,748.7562
20529,psiTPTE22|387590,6.7611,12.3336,398.9446,15.7143,69.5122,6.1112,17.6853,336.2314,17.0191,...,14.4928,13.8143,11.5801,12.0017,10.3578,13.6546,7.8878,10.6086,5.1765,0.0000


##### Taking log transformation of RNASeq to minimize skewness in the data

In [4]:
# Replacing all zeros with NaN value because log transform of zero is infinity. So I am replacing with NaN, it automatically ignores NaN value while doing log transformation
RNASeq = RNASeq.replace({'0.0000':np.nan, 0.0000:np.nan})
RNASeq

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0125-01A,TCGA-06-0125-02A,TCGA-06-0129-01A,TCGA-06-0130-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,NaN,NaN,NaN,NaN,NaN,0.4604,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,?|100133144,3.0732,NaN,2.9467,15.9952,5.5610,9.9246,3.3758,22.3207,0.6395,...,8.2279,15.2648,1.2190,12.2160,9.5716,2.4096,10.1709,8.4757,6.1600,NaN
2,?|100134869,6.7611,15.6973,13.9398,14.9571,4.8049,11.6684,3.3862,16.8239,2.9706,...,10.2174,13.8987,10.3611,17.3596,13.9689,6.4257,6.4812,3.2496,7.9576,9.9502
3,?|10357,54.7036,31.3945,60.3441,91.8238,62.5366,52.1491,41.6125,76.2289,35.0696,...,35.5731,27.6285,70.0838,108.1354,70.5556,43.0201,42.4584,62.2278,58.8188,60.1990
4,?|10431,886.3210,885.7744,1234.8285,470.0000,546.3415,391.9332,419.2458,642.3833,774.1104,...,512.5165,593.2464,507.6947,531.9331,588.0414,620.8835,533.7423,668.9001,458.3529,499.5025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,3867.9738,11481.4296,5066.8074,5090.9524,7800.0000,14516.6022,6251.2354,3838.6419,11495.6163,...,6793.1489,4724.4820,3173.5487,10100.3000,5263.6535,7469.0763,16936.8975,2922.3897,7212.2353,9608.4577
20527,ZZEF1|23140,1397.0927,606.0266,832.0844,1139.5238,627.4390,1350.9880,1396.6190,1963.4101,793.7081,...,948.6166,1670.7598,760.6278,752.6790,1092.7495,1244.1767,1024.9781,1862.6466,1561.4118,795.0249
20528,ZZZ3|26009,603.5834,718.1500,705.4354,700.4762,527.4390,730.0876,664.7594,1114.1786,900.4642,...,778.6561,599.3860,537.5590,910.4158,746.2335,541.3655,512.7082,632.6075,568.9412,748.7562
20529,psiTPTE22|387590,6.7611,12.3336,398.9446,15.7143,69.5122,6.1112,17.6853,336.2314,17.0191,...,14.4928,13.8143,11.5801,12.0017,10.3578,13.6546,7.8878,10.6086,5.1765,NaN


In [5]:
for column in RNASeq.columns:
    try:
        RNASeq[column] = np.log2(RNASeq[column].astype(float))
    except (ValueError, AttributeError):
        pass

RNASeq

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0125-01A,TCGA-06-0125-02A,TCGA-06-0129-01A,TCGA-06-0130-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,NaN,NaN,NaN,NaN,NaN,-1.119040,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,?|100133144,1.619742,NaN,1.559100,3.999567,2.475344,3.311009,1.755229,4.480310,-0.644984,...,3.040524,3.932137,0.285698,3.610700,3.258760,1.268794,3.346375,3.083333,2.622930,NaN
2,?|100134869,2.757258,3.972445,3.801138,3.902759,2.264506,3.544535,1.759667,4.072440,1.570754,...,3.352956,3.796878,3.373105,4.117662,3.804147,2.683854,2.696261,1.700262,2.992333,3.314726
3,?|10357,5.773564,4.972440,5.915141,6.520796,5.966629,5.704570,5.378945,6.252266,5.132149,...,5.152715,4.788085,6.131009,6.756695,6.140689,5.426939,5.407978,5.959487,5.878205,5.911668
4,?|10431,9.791685,9.790795,10.270095,8.876517,9.093659,8.614464,8.711653,9.327291,9.596396,...,9.001455,9.212488,8.987817,9.055101,9.199774,9.278179,9.060000,9.385647,8.840315,8.964348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,11.917362,13.487015,12.306861,12.313720,12.929258,13.825416,12.609926,11.906380,13.488796,...,12.729865,12.205940,11.631881,13.302111,12.361849,12.866714,14.047882,11.512933,12.816231,13.230089
20527,ZZEF1|23140,10.448212,9.243237,9.700586,10.154215,9.293331,10.399799,10.447723,10.939146,9.632465,...,9.889681,10.706289,9.571047,9.555891,10.093747,10.280976,10.001377,10.863138,10.608635,9.634856
20528,ZZZ3|26009,9.237409,9.488141,9.462370,9.452192,9.042860,9.511926,9.376688,10.121765,9.814525,...,9.604842,9.227342,9.070279,9.830382,9.543483,9.080459,9.001994,9.305167,9.152136,9.548352
20529,psiTPTE22|387590,2.757258,3.624522,8.640045,3.974006,6.119194,2.611456,4.144479,8.393311,4.089083,...,3.857264,3.788091,3.533576,3.585167,3.372646,3.771315,2.979623,3.407162,2.371977,NaN


In [6]:
# Again Replacing NaN with zeros.
RNASeq = RNASeq.replace({np.nan:0})
RNASeq

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0125-01A,TCGA-06-0125-02A,TCGA-06-0129-01A,TCGA-06-0130-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,0.000000,0.000000,0.000000,0.000000,0.000000,-1.119040,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,?|100133144,1.619742,0.000000,1.559100,3.999567,2.475344,3.311009,1.755229,4.480310,-0.644984,...,3.040524,3.932137,0.285698,3.610700,3.258760,1.268794,3.346375,3.083333,2.622930,0.000000
2,?|100134869,2.757258,3.972445,3.801138,3.902759,2.264506,3.544535,1.759667,4.072440,1.570754,...,3.352956,3.796878,3.373105,4.117662,3.804147,2.683854,2.696261,1.700262,2.992333,3.314726
3,?|10357,5.773564,4.972440,5.915141,6.520796,5.966629,5.704570,5.378945,6.252266,5.132149,...,5.152715,4.788085,6.131009,6.756695,6.140689,5.426939,5.407978,5.959487,5.878205,5.911668
4,?|10431,9.791685,9.790795,10.270095,8.876517,9.093659,8.614464,8.711653,9.327291,9.596396,...,9.001455,9.212488,8.987817,9.055101,9.199774,9.278179,9.060000,9.385647,8.840315,8.964348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,11.917362,13.487015,12.306861,12.313720,12.929258,13.825416,12.609926,11.906380,13.488796,...,12.729865,12.205940,11.631881,13.302111,12.361849,12.866714,14.047882,11.512933,12.816231,13.230089
20527,ZZEF1|23140,10.448212,9.243237,9.700586,10.154215,9.293331,10.399799,10.447723,10.939146,9.632465,...,9.889681,10.706289,9.571047,9.555891,10.093747,10.280976,10.001377,10.863138,10.608635,9.634856
20528,ZZZ3|26009,9.237409,9.488141,9.462370,9.452192,9.042860,9.511926,9.376688,10.121765,9.814525,...,9.604842,9.227342,9.070279,9.830382,9.543483,9.080459,9.001994,9.305167,9.152136,9.548352
20529,psiTPTE22|387590,2.757258,3.624522,8.640045,3.974006,6.119194,2.611456,4.144479,8.393311,4.089083,...,3.857264,3.788091,3.533576,3.585167,3.372646,3.771315,2.979623,3.407162,2.371977,0.000000


##### Exporting the prepared RNASeq data into csv file

In [7]:
RNASeq.to_csv('prepared/RNASeq.csv', index=False)

## 2. MutSigCV Dataset

##### Lets read the MutSigCV dataset

In [8]:
mut = pd.read_csv("data/GBM-TP/MutSigNozzleReport2CV.tsv", error_bad_lines=False, sep='\t', dtype='unicode')
mut

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_position,Strand,Variant_Classification,Variant_Type,...,categ_idx,trackpos,categ,gene,chr,pos,type,classification,ref_allele,patient
0,A1BG,1,broad.mit.edu,37,19,58863692,58863692,+,Silent,SNP,...,1,NaN,1,A1BG,19,58863692,Silent,SNP,G,TCGA-06-0744-01A-01W-0348-08
1,A1BG,1,broad.mit.edu,37,19,58858802,58858802,+,Missense_Mutation,SNP,...,3,98,3,A1BG,19,58858802,Missense_Mutation,SNP,C,TCGA-06-5411-01A-01D-1696-08
2,A1CF,29974,broad.mit.edu,37,10,52587910,52587910,+,Missense_Mutation,SNP,...,5,1062,5,A1CF,10,52587910,Missense_Mutation,SNP,T,TCGA-06-2558-01A-01D-1494-08
3,A1CF,29974,broad.mit.edu,37,10,52566580,52566580,+,Missense_Mutation,SNP,...,5,94,5,A1CF,10,52566580,Missense_Mutation,SNP,G,TCGA-14-0813-01A-01W-0424-08
4,A1CF,29974,broad.mit.edu,37,10,52596064,52596064,+,Missense_Mutation,SNP,...,1,1442,1,A1CF,10,52596064,Missense_Mutation,SNP,C,TCGA-27-1833-01A-01W-0643-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20233,ZXDC,79364,broad.mit.edu,37,3,126160628,126160628,+,Missense_Mutation,SNP,...,5,214,5,ZXDC,3,126160628,Missense_Mutation,SNP,C,TCGA-32-2491-01A-01D-1353-08
20234,ZYG11B,79699,broad.mit.edu,37,1,53250588,53250588,+,Missense_Mutation,SNP,...,3,1150,3,ZYG11B,1,53250588,Missense_Mutation,SNP,C,TCGA-19-5959-01A-11D-1696-08
20235,ZYX,7791,broad.mit.edu,37,7,143080299,143080299,+,Missense_Mutation,SNP,...,4,921,4,ZYX,7,143080299,Missense_Mutation,SNP,T,TCGA-12-3653-01A-01D-1495-08
20236,ZYX,7791,broad.mit.edu,37,7,143080252,143080252,+,Missense_Mutation,SNP,...,3,874,3,ZYX,7,143080252,Missense_Mutation,SNP,G,TCGA-76-4932-01A-01D-1486-08


##### Slicing Sample Id Barcode "TCGA-2G-AAEX-01A-11R-A430-07" from index 0 to index 16, and adding new column for sliced BarCode 'patient_sliced'

In [9]:
mut['patient_sliced'] = mut['patient'].str.slice(0, 16)
mut

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_position,Strand,Variant_Classification,Variant_Type,...,trackpos,categ,gene,chr,pos,type,classification,ref_allele,patient,patient_sliced
0,A1BG,1,broad.mit.edu,37,19,58863692,58863692,+,Silent,SNP,...,NaN,1,A1BG,19,58863692,Silent,SNP,G,TCGA-06-0744-01A-01W-0348-08,TCGA-06-0744-01A
1,A1BG,1,broad.mit.edu,37,19,58858802,58858802,+,Missense_Mutation,SNP,...,98,3,A1BG,19,58858802,Missense_Mutation,SNP,C,TCGA-06-5411-01A-01D-1696-08,TCGA-06-5411-01A
2,A1CF,29974,broad.mit.edu,37,10,52587910,52587910,+,Missense_Mutation,SNP,...,1062,5,A1CF,10,52587910,Missense_Mutation,SNP,T,TCGA-06-2558-01A-01D-1494-08,TCGA-06-2558-01A
3,A1CF,29974,broad.mit.edu,37,10,52566580,52566580,+,Missense_Mutation,SNP,...,94,5,A1CF,10,52566580,Missense_Mutation,SNP,G,TCGA-14-0813-01A-01W-0424-08,TCGA-14-0813-01A
4,A1CF,29974,broad.mit.edu,37,10,52596064,52596064,+,Missense_Mutation,SNP,...,1442,1,A1CF,10,52596064,Missense_Mutation,SNP,C,TCGA-27-1833-01A-01W-0643-08,TCGA-27-1833-01A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20233,ZXDC,79364,broad.mit.edu,37,3,126160628,126160628,+,Missense_Mutation,SNP,...,214,5,ZXDC,3,126160628,Missense_Mutation,SNP,C,TCGA-32-2491-01A-01D-1353-08,TCGA-32-2491-01A
20234,ZYG11B,79699,broad.mit.edu,37,1,53250588,53250588,+,Missense_Mutation,SNP,...,1150,3,ZYG11B,1,53250588,Missense_Mutation,SNP,C,TCGA-19-5959-01A-11D-1696-08,TCGA-19-5959-01A
20235,ZYX,7791,broad.mit.edu,37,7,143080299,143080299,+,Missense_Mutation,SNP,...,921,4,ZYX,7,143080299,Missense_Mutation,SNP,T,TCGA-12-3653-01A-01D-1495-08,TCGA-12-3653-01A
20236,ZYX,7791,broad.mit.edu,37,7,143080252,143080252,+,Missense_Mutation,SNP,...,874,3,ZYX,7,143080252,Missense_Mutation,SNP,G,TCGA-76-4932-01A-01D-1486-08,TCGA-76-4932-01A


##### MutSig2CV has many columns, most of them are not necessary for us. So filtering out columns which are not important.

In [10]:
mut = mut[['Hugo_Symbol', 'Entrez_Gene_Id', 'Variant_Classification', 'patient_sliced']]
mut.head()

,Hugo_Symbol,Entrez_Gene_Id,Variant_Classification,patient_sliced
0,A1BG,1,Silent,TCGA-06-0744-01A
1,A1BG,1,Missense_Mutation,TCGA-06-5411-01A
2,A1CF,29974,Missense_Mutation,TCGA-06-2558-01A
3,A1CF,29974,Missense_Mutation,TCGA-14-0813-01A
4,A1CF,29974,Missense_Mutation,TCGA-27-1833-01A


##### Now time to remove **5'UTR, Silent, Introns, 5'Flank, and 3'UTR** type of mutation from mutation dataset. These type of mutation are not significant so we can remove these mutation from datataset

In [11]:
mut.drop(mut[mut.Variant_Classification == '5\'UTR'].index, inplace=True)
mut.drop(mut[mut.Variant_Classification == 'Silent'].index, inplace=True)
mut.drop(mut[mut.Variant_Classification == 'Intron'].index, inplace=True)
mut.drop(mut[mut.Variant_Classification == '3\'UTR'].index, inplace=True)
mut.drop(mut[mut.Variant_Classification == '5\'Flank'].index, inplace=True)
mut

,Hugo_Symbol,Entrez_Gene_Id,Variant_Classification,patient_sliced
1,A1BG,1,Missense_Mutation,TCGA-06-5411-01A
2,A1CF,29974,Missense_Mutation,TCGA-06-2558-01A
3,A1CF,29974,Missense_Mutation,TCGA-14-0813-01A
4,A1CF,29974,Missense_Mutation,TCGA-27-1833-01A
5,A2BP1,54715,Missense_Mutation,TCGA-06-0137-01A
...,...,...,...,...
20233,ZXDC,79364,Missense_Mutation,TCGA-32-2491-01A
20234,ZYG11B,79699,Missense_Mutation,TCGA-19-5959-01A
20235,ZYX,7791,Missense_Mutation,TCGA-12-3653-01A
20236,ZYX,7791,Missense_Mutation,TCGA-76-4932-01A


##### There are 283 unique patient Id's in mutation dataset but RNASeq dataset has only 141 sample Ids. So removing all Sample Ids from mutation datasets which are not available in RNASeq Dataset.

In [12]:
a = mut.patient_sliced.unique()

In [13]:
b = RNASeq.columns.unique()
intersectIDs = (np.intersect1d(a,b))
mut = mut[mut['patient_sliced'].isin(intersectIDs)]
mut

,Hugo_Symbol,Entrez_Gene_Id,Variant_Classification,patient_sliced
1,A1BG,1,Missense_Mutation,TCGA-06-5411-01A
2,A1CF,29974,Missense_Mutation,TCGA-06-2558-01A
8,A2BP1,54715,Missense_Mutation,TCGA-28-5207-01A
15,AACS,65985,Missense_Mutation,TCGA-14-1825-01A
17,AACS,65985,Frame_Shift_Del,TCGA-76-4926-01B
...,...,...,...,...
20230,ZXDA,7789,Missense_Mutation,TCGA-41-5651-01A
20232,ZXDB,158586,Missense_Mutation,TCGA-06-0745-01A
20235,ZYX,7791,Missense_Mutation,TCGA-12-3653-01A
20236,ZYX,7791,Missense_Mutation,TCGA-76-4932-01A


In [14]:
RNASeq = RNASeq[RNASeq.columns.intersection(np.append(intersectIDs, 'HybridizationREF'))]
RNASeq.to_csv('prepared/RNASeq.csv', index=False)
RNASeq

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0129-01A,TCGA-06-0130-01A,TCGA-06-0132-01A,TCGA-06-0141-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,?|100133144,1.619742,0.000000,1.559100,3.999567,2.475344,4.480310,-0.644984,1.014712,2.358481,...,3.040524,3.932137,0.285698,3.610700,3.258760,1.268794,3.346375,3.083333,2.622930,0.000000
2,?|100134869,2.757258,3.972445,3.801138,3.902759,2.264506,4.072440,1.570754,1.768798,3.126312,...,3.352956,3.796878,3.373105,4.117662,3.804147,2.683854,2.696261,1.700262,2.992333,3.314726
3,?|10357,5.773564,4.972440,5.915141,6.520796,5.966629,6.252266,5.132149,5.288702,5.943574,...,5.152715,4.788085,6.131009,6.756695,6.140689,5.426939,5.407978,5.959487,5.878205,5.911668
4,?|10431,9.791685,9.790795,10.270095,8.876517,9.093659,9.327291,9.596396,9.471455,9.766777,...,9.001455,9.212488,8.987817,9.055101,9.199774,9.278179,9.060000,9.385647,8.840315,8.964348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,11.917362,13.487015,12.306861,12.313720,12.929258,11.906380,13.488796,12.484875,12.721659,...,12.729865,12.205940,11.631881,13.302111,12.361849,12.866714,14.047882,11.512933,12.816231,13.230089
20527,ZZEF1|23140,10.448212,9.243237,9.700586,10.154215,9.293331,10.939146,9.632465,9.510155,10.207305,...,9.889681,10.706289,9.571047,9.555891,10.093747,10.280976,10.001377,10.863138,10.608635,9.634856
20528,ZZZ3|26009,9.237409,9.488141,9.462370,9.452192,9.042860,10.121765,9.814525,9.084337,9.469504,...,9.604842,9.227342,9.070279,9.830382,9.543483,9.080459,9.001994,9.305167,9.152136,9.548352
20529,psiTPTE22|387590,2.757258,3.624522,8.640045,3.974006,6.119194,8.393311,4.089083,6.592484,6.295394,...,3.857264,3.788091,3.533576,3.585167,3.372646,3.771315,2.979623,3.407162,2.371977,0.000000


##### Exporting the prepared Mutation data into csv file

In [15]:
mut.to_csv('prepared/Mutation.csv', index=False)
mut

,Hugo_Symbol,Entrez_Gene_Id,Variant_Classification,patient_sliced
1,A1BG,1,Missense_Mutation,TCGA-06-5411-01A
2,A1CF,29974,Missense_Mutation,TCGA-06-2558-01A
8,A2BP1,54715,Missense_Mutation,TCGA-28-5207-01A
15,AACS,65985,Missense_Mutation,TCGA-14-1825-01A
17,AACS,65985,Frame_Shift_Del,TCGA-76-4926-01B
...,...,...,...,...
20230,ZXDA,7789,Missense_Mutation,TCGA-41-5651-01A
20232,ZXDB,158586,Missense_Mutation,TCGA-06-0745-01A
20235,ZYX,7791,Missense_Mutation,TCGA-12-3653-01A
20236,ZYX,7791,Missense_Mutation,TCGA-76-4932-01A


# 3. Output Matrix

##### Creating output matrix of genes X samples.  The output matrix is a matrix of each gene is mutated or not in a given sample id

In [16]:
# Here combining Gene symbol and gene ID to create new column i.e. HybridizationREF
mut['HybridizationREF'] = mut['Hugo_Symbol'] + "|" + mut['Entrez_Gene_Id']
mut

,Hugo_Symbol,Entrez_Gene_Id,Variant_Classification,patient_sliced,HybridizationREF
1,A1BG,1,Missense_Mutation,TCGA-06-5411-01A,A1BG|1
2,A1CF,29974,Missense_Mutation,TCGA-06-2558-01A,A1CF|29974
8,A2BP1,54715,Missense_Mutation,TCGA-28-5207-01A,A2BP1|54715
15,AACS,65985,Missense_Mutation,TCGA-14-1825-01A,AACS|65985
17,AACS,65985,Frame_Shift_Del,TCGA-76-4926-01B,AACS|65985
...,...,...,...,...,...
20230,ZXDA,7789,Missense_Mutation,TCGA-41-5651-01A,ZXDA|7789
20232,ZXDB,158586,Missense_Mutation,TCGA-06-0745-01A,ZXDB|158586
20235,ZYX,7791,Missense_Mutation,TCGA-12-3653-01A,ZYX|7791
20236,ZYX,7791,Missense_Mutation,TCGA-76-4932-01A,ZYX|7791


In [17]:
# Creating empty dataframe
y = pd.DataFrame(columns=RNASeq.columns)
y.HybridizationREF = RNASeq['HybridizationREF']
y

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0129-01A,TCGA-06-0130-01A,TCGA-06-0132-01A,TCGA-06-0141-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,?|100133144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,?|100134869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,?|10357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,?|10431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20527,ZZEF1|23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20528,ZZZ3|26009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20529,psiTPTE22|387590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Now checking each gene of each sample in **RNASEQ dataset** and inputing 0 if it is not exist in **mutation dataset** and 1 if it is exist.

In [18]:
for sample in RNASeq.columns:
    if sample == "HybridizationREF":
        continue
    a = mut[mut['patient_sliced'] == sample]
    u = RNASeq['HybridizationREF'].isin(a['HybridizationREF'])
    y[sample] = u
    y[sample] = y[sample].replace({True: 1, False: 0})
y

,HybridizationREF,TCGA-02-0047-01A,TCGA-02-0055-01A,TCGA-02-2483-01A,TCGA-02-2485-01A,TCGA-02-2486-01A,TCGA-06-0129-01A,TCGA-06-0130-01A,TCGA-06-0132-01A,TCGA-06-0141-01A,...,TCGA-41-3915-01A,TCGA-41-4097-01A,TCGA-41-5651-01A,TCGA-76-4925-01A,TCGA-76-4926-01B,TCGA-76-4927-01A,TCGA-76-4928-01B,TCGA-76-4929-01A,TCGA-76-4931-01A,TCGA-76-4932-01A
0,?|100130426,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,?|100133144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,?|100134869,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,?|10357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,?|10431,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYX|7791,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
20527,ZZEF1|23140,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20528,ZZZ3|26009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20529,psiTPTE22|387590,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
y.to_csv('prepared/y.csv', index=False)